In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import os
import gc
import numpy as np
import pandas as pd



In [2]:
import importlib
import utils
importlib.reload(utils)


print(dir(utils))
from utils import extract_division_names
from utils import siren_filter
from utils import compute_null_values
from utils import var_filter
from utils import count_target_value





['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'compute_null_values', 'count_nb_lines', 'count_target_value', 'extract_division_names', 'pa', 'pc', 'pq', 'siren_filter', 'var_filter']


In [3]:
os.makedirs("datasets_temp", exist_ok=True)
os.makedirs("datasets_finaux", exist_ok=True)

Etant donné que l'utilisation de la mémoire tout au long du notebook est très importante (risquant de faire crasher le logiciel), on libère la mémoire allouée aux variables gérant les dataframes une fois qu'elles ne sont plus utilisées dans la suite.

In [4]:
#gestion mémoire  (l'équivalent de free en C)

gc.set_threshold(500, 5, 5)
gc.collect()

0

Préparation du bilan administratif (contient les labels, les dates de création et autres informations administratives)

In [5]:
bilan_admin=pq.ParquetFile("datasets_originaux/StockUniteLegale_utf8.parquet")


In [6]:
#NOMS DES COLONNES + APPERCU DE QUELQUES LIGNES
groupe=bilan_admin.read_row_group(0)    #pour ne pas faire crasher le logiciel, on regarde les datasets groupe par groupe (ici c'est le groupe 0)

df=groupe.to_pandas()


print(df.columns)
df.head(10)

Index(['siren', 'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'sigleUniteLegale', 'sexeUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale', 'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale',
       'anneeEffectifsUniteLegale', 'dateDernierTraitementUniteLegale',
       'nombrePeriodesUniteLegale', 'categorieEntreprise',
       'anneeCategorieEntreprise', 'dateDebut', 'etatAdministratifUniteLegale',
       'nomUniteLegale', 'nomUsageUniteLegale', 'denominationUniteLegale',
       'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'categorieJuridiqueUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale', 'nicSiegeUniteLegale',
       'economieSocialeSolidaireUniteLegale', 'societeMissionUniteLegale',

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,...,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale,activitePrincipaleNAF25UniteLegale
0,000325175,O,None,2000-09-26,None,M,THIERRY,None,None,None,...,None,None,1000,32.12Z,NAFRev2,65,None,None,None,32.12Y
1,001807254,O,None,1972-05-01,None,M,JACQUES-LUCIEN,None,None,None,...,None,None,1000,85.59A,NAFRev2,22,None,None,None,None
2,005410220,O,True,1954-12-25,None,M,GEORGES,None,None,None,...,None,None,1000,22.02,NAP,15,None,None,None,None
3,005410345,O,True,None,None,M,MICHEL,None,None,None,...,None,None,1000,79.06,NAP,10,None,None,None,None
4,005410394,O,True,1954-12-25,None,M,ROBERT,ALFRED,None,None,...,None,None,1000,64.42,NAP,18,None,None,None,None
5,005410428,O,True,1954-01-01,None,M,RENE,PIERRE,None,None,...,None,None,1000,70.2C,NAF1993,14,None,None,None,None
6,005410436,O,True,None,None,M,MARCEL,None,None,None,...,None,None,1000,57.11,NAP,17,None,None,None,None
7,005410485,O,True,None,None,M,RENE,ALFRED,None,None,...,None,None,1000,64.42,NAP,14,None,None,None,None
8,005410493,O,True,1954-12-25,None,M,PIERRE,None,None,None,...,None,None,1000,86.06,NAP,18,None,None,None,None
9,005410501,O,True,1954-01-01,None,F,ANDREE,None,None,None,...,None,None,1000,55.4B,NAF1993,18,None,None,None,None


In [7]:
#NOMBRE + PROPORTION D'ENTREPRISES SURVIVANTES (Toutes entreprises confondues)

nb_survivants,proportion_survivants=count_target_value("A", 'etatAdministratifUniteLegale',bilan_admin, proportion=True)
print(f"Nombre de survivants : {nb_survivants} ")
print(f"Proportion de survivants : {proportion_survivants}")

Nombre de survivants : 16931566 
Proportion de survivants : 0.5829311778603316


In [8]:
#colonnes interessantes (on enlève les variables type prénom, nom, nomenclature, etc...)

columns=['siren', 
        'dateCreationUniteLegale',
        'etatAdministratifUniteLegale',
        'trancheEffectifsUniteLegale',
        'anneeEffectifsUniteLegale', 
        'categorieEntreprise',
        'anneeCategorieEntreprise',
        'categorieJuridiqueUniteLegale', 
        'activitePrincipaleUniteLegale', 
        'economieSocialeSolidaireUniteLegale',
        'societeMissionUniteLegale',
        'caractereEmployeurUniteLegale',
        'nombrePeriodesUniteLegale'
        ]


In [9]:
#COMPTER LE NOMBRE DE VALEURS NULL ET PROPORTION DE VALEURS NULL POUR CHAQUE COLONNE

null_values,null_values_proportion=compute_null_values(columns, bilan_admin, proportion=True)


print("Proportion de valeurs null :")
for i in range(len(null_values_proportion)):
    print(f"    {columns[i]} : {null_values_proportion[i]}")




Proportion de valeurs null :
    siren : 0.0
    dateCreationUniteLegale : 0.0403580360596175
    etatAdministratifUniteLegale : 0.0
    trancheEffectifsUniteLegale : 0.0
    anneeEffectifsUniteLegale : 0.9308428694417592
    categorieEntreprise : 0.6369077469518067
    anneeCategorieEntreprise : 0.6369077469518067
    categorieJuridiqueUniteLegale : 0.0
    activitePrincipaleUniteLegale : 0.0007410425398492837
    economieSocialeSolidaireUniteLegale : 0.7852676033236881
    societeMissionUniteLegale : 0.968980635460848
    caractereEmployeurUniteLegale : 1.0
    nombrePeriodesUniteLegale : 0.0


In [10]:
#FILTRE DES SECTEURS/CATEGORIES POUR NE GARDER QUE LES STARTUPS

#on commence d'abord par filtrer avec ces critères pour avoir un dataset plus léger et ainsi mieux le gérer


categories_juridiques=["5710"]
secteurs_innovants= [
    # Numérique / Logiciel
    "58.21Z",  # Édition de jeux électroniques
    "58.29A",  # Édition de logiciels système et réseau
    "58.29B",  # Édition de logiciels applicatifs
    "62.01Z",  # Programmation informatique
    "62.02A",  # Conseil en systèmes et logiciels informatiques
    "62.02B",  # Tierce maintenance de systèmes et d’applications
    "62.09Z",  # Autres activités informatiques

    # Données / plateformes / cloud
    "63.11Z",  # Traitement de données, hébergement et activités connexes
    "63.12Z",  # Portails Internet
    "63.99Z",  # Autres services d’information n.c.a.

    # R&D / Deeptech
    "72.11Z",  # R&D en biotechnologie
    "72.19Z",  # R&D en autres sciences physiques et naturelles

    # Industrie technologique / hardware
    "26.11Z",  # Fabrication de composants électroniques
    "26.20Z",  # Fabrication d’ordinateurs et d’équipements périphériques
    "26.51A",  # Fabrication d’instruments de mesure
    "26.51B",  # Fabrication	d'instrumentation	scientifique	et	technique

    # Conseil technologique à forte intensité innovation
    "70.22Z",  # Conseil pour les affaires et autres conseils de gestion (tech / SaaS)
]

var_filter(bilan_admin,'activitePrincipaleUniteLegale',"datasets_temp/bilan_admin_v2.parquet",secteurs_innovants)
bilan_admin=pq.ParquetFile("datasets_temp/bilan_admin_v2.parquet")

var_filter(bilan_admin,  'categorieJuridiqueUniteLegale',"datasets_temp/bilan_admin_v3.parquet",categories_juridiques)

#liberer memoire
gc.collect()

0

In [11]:
bilan_admin=pq.ParquetFile("datasets_temp/bilan_admin_v3.parquet")

In [12]:
df=pd.read_parquet("datasets_temp/bilan_admin_v3.parquet")
print(df.shape)
print(df.columns)
df.head(10)

(285441, 35)
Index(['siren', 'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'sigleUniteLegale', 'sexeUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale', 'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale',
       'anneeEffectifsUniteLegale', 'dateDernierTraitementUniteLegale',
       'nombrePeriodesUniteLegale', 'categorieEntreprise',
       'anneeCategorieEntreprise', 'dateDebut', 'etatAdministratifUniteLegale',
       'nomUniteLegale', 'nomUsageUniteLegale', 'denominationUniteLegale',
       'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'categorieJuridiqueUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale', 'nicSiegeUniteLegale',
       'economieSocialeSolidaireUniteLegale', 'societeMission

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,...,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale,activitePrincipaleNAF25UniteLegale
0,005580436,O,None,1955-01-01,None,None,None,None,None,None,...,None,None,5710,70.22Z,NAFRev2,60,N,None,None,70.20Y
1,024074924,O,None,2002-02-28,T.D.I.,None,None,None,None,None,...,None,None,5710,63.11Z,NAFRev2,10,N,None,None,63.10Y
2,035650167,O,None,1956-01-01,None,None,None,None,None,None,...,None,None,5710,26.11Z,NAFRev2,89,None,None,None,26.11Y
3,036420040,O,None,1964-01-01,TIF,None,None,None,None,None,...,None,None,5710,72.19Z,NAFRev2,291,N,None,None,72.10H
4,046750014,O,None,1967-01-01,None,None,None,None,None,None,...,None,None,5710,72.11Z,NAFRev2,22,N,None,None,72.10G
5,055811095,O,None,1955-01-01,SAGA,None,None,None,None,None,...,None,None,5710,70.22Z,NAFRev2,45,N,None,None,None
6,057801136,O,None,1957-01-01,None,None,None,None,None,None,...,None,None,5710,26.51A,NAFRev2,11,None,None,None,None
7,062804414,O,None,1962-01-01,None,None,None,None,None,None,...,None,None,5710,70.22Z,NAFRev2,118,N,None,None,None
8,064502636,O,None,1964-01-01,None,None,None,None,None,None,...,None,None,5710,62.02A,NAFRev2,679,N,None,None,None
9,067201087,O,None,1967-01-01,None,None,None,None,None,None,...,None,None,5710,70.22Z,NAFRev2,37,N,None,None,70.20Y


In [13]:
#SUPPRESSION DES COLONNES ININTERESSANTES (noms, prénoms, nomenclature, colonnes vides à 100%)
df=df.drop(['statutDiffusionUniteLegale', 
        'unitePurgeeUniteLegale',
        'sigleUniteLegale',
        'sexeUniteLegale',
       'prenom1UniteLegale',
        
         'prenom2UniteLegale',
           'prenom3UniteLegale',
       'prenom4UniteLegale', 
       'prenomUsuelUniteLegale',
         'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale',
        'dateDernierTraitementUniteLegale',
        'dateDebut',
        'nomUniteLegale',
          'nomUsageUniteLegale',
            'denominationUniteLegale',
       'denominationUsuelle1UniteLegale',
         'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
         'nicSiegeUniteLegale',
         'caractereEmployeurUniteLegale',
         'activitePrincipaleNAF25UniteLegale'
       ], axis=1)

In [14]:
#SUPPRESSION DES LIGNES NULL POUR LE LABEL, SIREN ET DATE DE CREATION

df=df.dropna(subset=['siren'])
df = df.dropna(subset=['etatAdministratifUniteLegale'])
df = df.dropna(subset=['dateCreationUniteLegale'])


In [15]:
#OBSERVATION DES DIFFERENTES VALEURS PRISES PAR CERTAINES COLONNES

categorieEntreprise_vals=df["categorieEntreprise"].unique()
print(f"categorieEntreprise_vals : {categorieEntreprise_vals}")

trancheEffectifsUniteLegale=df["trancheEffectifsUniteLegale"].unique()
print(f"trancheEffectifsUniteLegale : {trancheEffectifsUniteLegale}")

anneeEffectifsUniteLegale=df["anneeEffectifsUniteLegale"].unique()
print(f"anneeEffectifsUniteLegale : {anneeEffectifsUniteLegale}")

anneeCategorieEntreprise=df["anneeCategorieEntreprise"].unique()
print(f"anneeCategorieEntreprise : {anneeCategorieEntreprise}")


economieSocialeSolidaireUniteLegale=df["economieSocialeSolidaireUniteLegale"].unique()
print(f"economieSocialeSolidaireUniteLegale : {economieSocialeSolidaireUniteLegale}")

societeMissionUniteLegale=df["societeMissionUniteLegale"].unique()
print(f"societeMissionUniteLegale : {societeMissionUniteLegale}")

etatAdministratifUniteLegale=df["etatAdministratifUniteLegale"].unique()
print(f"etatAdministratifUniteLegale : {etatAdministratifUniteLegale}")

nombre_periodes=df['nombrePeriodesUniteLegale'].unique()
print(f"nombrePeriodesUniteLegale : {nombre_periodes}")

categorieEntreprise_vals : ['PME' 'ETI' None 'GE']
trancheEffectifsUniteLegale : ['NN' '32' '12' '21' '02' '11' '00' '01' '03' '41' '22' '31' '42' '52'
 '51' '53']
anneeEffectifsUniteLegale : [  nan 2023.]
anneeCategorieEntreprise : [2023.   nan]
economieSocialeSolidaireUniteLegale : ['N' None 'O']
societeMissionUniteLegale : [None 'N' 'O']
etatAdministratifUniteLegale : ['A' 'C']
nombrePeriodesUniteLegale : [ 7  2  6  8  5  9 12  3 10  4 16 11 17 15 14 13 18 26 19 22  1]


In [16]:
#MAPPING DES VARIABLES CATEGORIELLES

dictNO={'N':-1, 'O':1}
dictcategorieentreprise={'ETI':2, 'GE':3, 'PME':1}
dictAC={'A':1 , 'C':0}

In [17]:
#MAPPING POUR N'AVOIR PLUS DE NULL

df['categorieEntreprise'] = df['categorieEntreprise'].map(dictcategorieentreprise).fillna(0).astype(int)
df['economieSocialeSolidaireUniteLegale'] = df['economieSocialeSolidaireUniteLegale'].map(dictNO).fillna(0).astype(int)
df['societeMissionUniteLegale'] = df['societeMissionUniteLegale'].map(dictNO).fillna(0).astype(int)
df['anneeEffectifsUniteLegale']=df['anneeEffectifsUniteLegale'].fillna(0).astype(int)
df['anneeCategorieEntreprise']=df['anneeCategorieEntreprise'].fillna(0).astype(int)
df['etatAdministratifUniteLegale']=df['etatAdministratifUniteLegale'].map(dictAC).astype(int)

df.head(20)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,005580436,1955-01-01,NN,0,7,1,2023,1,5710,70.22Z,-1,0
1,024074924,2002-02-28,NN,0,2,2,2023,1,5710,63.11Z,-1,0
2,035650167,1956-01-01,32,2023,6,2,2023,1,5710,26.11Z,0,0
3,036420040,1964-01-01,12,2023,8,1,2023,1,5710,72.19Z,-1,0
4,046750014,1967-01-01,12,2023,5,1,2023,1,5710,72.11Z,-1,0
5,055811095,1955-01-01,NN,0,7,0,0,0,5710,70.22Z,-1,0
6,057801136,1957-01-01,NN,0,5,0,0,0,5710,26.51A,0,0
7,062804414,1962-01-01,NN,0,9,1,2023,0,5710,70.22Z,-1,0
8,064502636,1964-01-01,NN,0,12,0,0,0,5710,62.02A,-1,0
9,067201087,1967-01-01,NN,0,7,2,2023,1,5710,70.22Z,-1,0


In [18]:
#CONVERSION DATE DE CREATION EN ANNEE DE CREATION


df['dateCreationUniteLegale'] = pd.to_datetime(df['dateCreationUniteLegale'], errors='coerce')
df['dateCreationUniteLegale'] = df['dateCreationUniteLegale'].dt.year.astype(int)

In [19]:
#CONVERSION DE TRANCHE EFFECTIF EN ENTIER

def tranche_to_int(val):
    if val == 'NN' or pd.isna(val):
        return -1
    else:
        return int(val)

df["trancheEffectifsUniteLegale"] = df['trancheEffectifsUniteLegale'].apply(tranche_to_int)
df["trancheEffectifsUniteLegale"].head(5)

0    -1
1    -1
2    32
3    12
4    12
Name: trancheEffectifsUniteLegale, dtype: int64

In [20]:
#IDEM POUR ACTIVITE PRINCIPALE (ON GARDE QUE LES 4 PREMIERS CHIFFRES)

def activite_to_int(val):
    return int(val[:2] + val[3:5])

df['activitePrincipaleUniteLegale'] = df['activitePrincipaleUniteLegale'].apply(activite_to_int)
df["siren"] = df["siren"].astype(int)

In [21]:
#VERIFICATION TYPE POUR N'AVOIR QUE DES ENTIERS
print(df.dtypes)
df.head(10)


siren                                  int64
dateCreationUniteLegale                int64
trancheEffectifsUniteLegale            int64
anneeEffectifsUniteLegale              int64
nombrePeriodesUniteLegale              int64
categorieEntreprise                    int64
anneeCategorieEntreprise               int64
etatAdministratifUniteLegale           int64
categorieJuridiqueUniteLegale          int64
activitePrincipaleUniteLegale          int64
economieSocialeSolidaireUniteLegale    int64
societeMissionUniteLegale              int64
dtype: object


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0


In [22]:
#RECHERCHE DE VALEURS NULL

df.isna().sum().sum()

np.int64(0)

In [23]:
#CONVERSION EN FICHIER PARQUET (SAUVEGARDE)

df.to_parquet("datasets_temp/bilan_admin_v4.parquet", engine='pyarrow', index=False)


#MEMOIRE LIBEREE
del df
gc.collect()

0

Gestion de fichier sur les données d'établissements (une entreprise peut avoir plusieurs établissements)

In [24]:
bilan_etablissement=pq.ParquetFile("datasets_originaux/StockEtablissement_utf8.parquet")


In [25]:
group = bilan_etablissement.read_row_groups([0])  #lit le premier row group
df = group.to_pandas()
print(df.columns)
df.head(10)


Index(['siren', 'nic', 'siret', 'statutDiffusionEtablissement',
       'dateCreationEtablissement', 'trancheEffectifsEtablissement',
       'anneeEffectifsEtablissement',
       'activitePrincipaleRegistreMetiersEtablissement',
       'dateDernierTraitementEtablissement', 'etablissementSiege',
       'nombrePeriodesEtablissement', 'complementAdresseEtablissement',
       'numeroVoieEtablissement', 'indiceRepetitionEtablissement',
       'dernierNumeroVoieEtablissement',
       'indiceRepetitionDernierNumeroVoieEtablissement',
       'typeVoieEtablissement', 'libelleVoieEtablissement',
       'codePostalEtablissement', 'libelleCommuneEtablissement',
       'libelleCommuneEtrangerEtablissement',
       'distributionSpecialeEtablissement', 'codeCommuneEtablissement',
       'codeCedexEtablissement', 'libelleCedexEtablissement',
       'codePaysEtrangerEtablissement', 'libellePaysEtrangerEtablissement',
       'identifiantAdresseEtablissement',
       'coordonneeLambertAbscisseEtablissemen

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,activitePrincipaleNAF25Etablissement
0,000325175,16,00032517500016,O,2000-09-26,NN,NaN,3212ZZ,2024-03-22 15:40:57,False,...,2009-05-27,F,None,None,None,None,32.12Z,NAFRev2,N,None
1,000325175,24,00032517500024,O,2008-05-20,NN,NaN,None,2024-03-30 05:08:28,False,...,2011-10-21,F,TAHITI PERLES CREATIONS,None,None,None,47.89Z,NAFRev2,N,None
2,000325175,32,00032517500032,O,2009-05-27,NN,NaN,None,2024-03-30 05:56:39,False,...,2011-10-21,F,TAHITI PERLES CREATIONS,None,None,None,32.12Z,NAFRev2,N,None
3,000325175,40,00032517500040,O,2011-10-21,NN,NaN,3212ZZ,2024-03-30 05:48:29,False,...,2014-01-07,F,TAHITI PERLES CREATION,None,None,None,32.12Z,NAFRev2,N,None
4,000325175,57,00032517500057,O,2014-01-07,NN,NaN,None,2024-03-30 05:29:10,False,...,2018-02-07,F,TAHITI PERLES CREATION,None,None,None,32.12Z,NAFRev2,N,None
5,000325175,65,00032517500065,O,2018-02-07,NN,NaN,None,2025-12-06 00:08:31,True,...,2018-02-07,A,None,None,None,None,32.12Z,NAFRev2,N,32.12Y
6,001807254,14,00180725400014,O,1972-05-01,NN,NaN,None,2024-03-30 03:53:05,False,...,2008-01-01,F,None,None,None,None,95.24Z,NAFRev2,N,None
7,001807254,22,00180725400022,O,2009-10-01,NN,NaN,None,2024-03-30 03:53:05,True,...,2014-12-31,F,None,None,None,None,85.59A,NAFRev2,N,None
8,005410220,15,00541022000015,O,None,NN,NaN,None,2024-03-22 15:40:57,True,...,1988-03-31,F,None,None,None,None,22.02,NAP,N,None
9,005410345,10,00541034500010,O,None,NN,NaN,None,2024-03-30 16:08:33,True,...,1984-12-25,F,None,None,None,None,79.06,NAP,N,None


In [26]:
#FILTRER (ON NE GARDE QUE LES SIRENS PRESENTES DANS LE DATASET DES DONNEES ADMINISTRATIVES)
df_admin = pd.read_parquet("datasets_temp/bilan_admin_v4.parquet", columns=["siren"])

sirens_retenus = set(df_admin["siren"])

del df_admin
gc.collect()

siren_filter(bilan_etablissement, "siren", "datasets_temp/bilan_etablissement_v2.parquet", sirens_retenus)

del sirens_retenus
gc.collect()

0

In [27]:
bilan_etablissement=pq.ParquetFile("datasets_temp/bilan_etablissement_v2.parquet")

In [28]:

#CREATION DE NOUVELLE VARIABLE (NOMBRE D'ETABLISSEMENTS POUR CHAQUE STARTUP)

dict_siren_nb_etablissements={}

for i in range(bilan_etablissement.num_row_groups):
    groupe=bilan_etablissement.read_row_group(i,columns=["siren",'etatAdministratifEtablissement'])
    sirens = groupe.column("siren").to_pylist()
    
    etats=groupe.column('etatAdministratifEtablissement').to_pylist()
    for siren in sirens:

        if siren not in dict_siren_nb_etablissements:
            dict_siren_nb_etablissements[siren] = 0
        dict_siren_nb_etablissements[siren] +=1

    #gestion de mémoire (on libère la mémoire tous les 20 groupes)
    del groupe, sirens, etats
    if (i + 1) % 20 == 0:
        gc.collect()


gc.collect()

0

In [29]:
sirens = list(dict_siren_nb_etablissements.keys())
nb_etablissements = [dict_siren_nb_etablissements[s] for s in sirens]

print(f"Nombre de startups : {len(sirens)}")
print(f"Nombre d'établissements moyen par startup : {np.mean(nb_etablissements)}")


Nombre de startups : 285441
Nombre d'établissements moyen par startup : 1.5496407313595455


In [30]:
#ECRITURE DES NOUVELLES COLONNES

CHUNK_SIZE = 100_000  

output_file = "datasets_temp/bilan_etablissement_v3.parquet"
schema = pa.schema([('siren', pa.int64()),('nbEtablissements', pa.int32())])

writer = pq.ParquetWriter(output_file, schema, compression='snappy')

list_sirens = list(dict_siren_nb_etablissements.keys())
nb_sirens = len(list_sirens)

for i in range(0, nb_sirens, CHUNK_SIZE):
    end = min(i + CHUNK_SIZE, nb_sirens)
    chunk = list_sirens[i:end]
    
    table = pa.Table.from_arrays([chunk,[dict_siren_nb_etablissements[s] for s in chunk],],schema=schema)
    
    writer.write_table(table)
    
    if (i // CHUNK_SIZE) % 10 == 0:
        gc.collect()

writer.close()

del dict_siren_nb_etablissements, 
del list_sirens, table, writer
gc.collect()


0

In [31]:
df_etablissement=pd.read_parquet("datasets_temp/bilan_etablissement_v3.parquet")
df_admin=pd.read_parquet("datasets_temp/bilan_admin_v4.parquet")

In [32]:
siren_etab = set(df_etablissement["siren"].astype(str))
siren_admin = set(df_admin["siren"].astype(str))

print(siren_etab-siren_admin) #rechercher si il y a des différences en termes de sirens

set()


In [33]:
df_admin.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0


In [34]:
df_etablissement.head(10)

,siren,nbEtablissements
0,5580436,3
1,24074924,1
2,35650167,6
3,36420040,16
4,46750014,3
5,55811095,3
6,57801136,1
7,62804414,5
8,64502636,50
9,67201087,2


In [35]:
#FUSION ENTRE DATASET ADMINISTRATIF ET DATASET DES ETABLISSEMENTS

df_adminetab = df_admin.merge(df_etablissement, on='siren', how='left')


del df_admin, df_etablissement
gc.collect()

df_adminetab.shape

(285441, 13)

In [36]:
df_adminetab.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2


In [37]:
df_adminetab.to_parquet('datasets_temp/bilan_adminetab.parquet',engine='pyarrow',compression='snappy',index=False) #sauvegarde du fichier



Gestion du dataset sur les bilans financiers

In [38]:
bilan_financier=pq.ParquetFile("datasets_originaux/ratios_inpi_bce.parquet")
bilan_adminetab=pq.ParquetFile("datasets_temp/bilan_adminetab.parquet")



In [39]:

#extraire les sirens uniques
sirens_bilan_adminetab=extract_division_names(bilan_adminetab,"siren")
sirens_bilan_financier=extract_division_names(bilan_financier,"siren")


sirens_bilan_financier_list = sorted(sirens_bilan_financier) 
print(f"Nombre d'entreprises ayant déclaré un bilan financier : {len(sirens_bilan_financier_list)}")


Nombre d'entreprises ayant déclaré un bilan financier : 1542770


In [40]:
df_adminetab = pd.read_parquet("datasets_temp/bilan_adminetab.parquet")

sirens_retenus = set(df_adminetab["siren"])

gc.collect()


siren_filter(bilan_financier, "siren", "datasets_temp/bilan_financier_v2.parquet", sirens_retenus)

del sirens_retenus
gc.collect()

0

In [41]:
df_financier=pd.read_parquet("datasets_temp/bilan_financier_v2.parquet")

In [42]:
df_financier.shape[0]

221632

In [43]:
df_financier = pd.read_parquet("datasets_temp/bilan_financier_v2.parquet")
print(df_financier.shape)
df_financier.columns

(221632, 23)


Index(['siren', 'date_cloture_exercice', 'Chiffre_d_affaires', 'Marge_brute',
       'EBE', 'EBIT', 'Resultat_net', 'Taux_d_endettement',
       'Ratio_de_liquidite', 'Ratio_de_vetuste', 'Autonomie_financiere',
       'Poids_BFR_exploitation_sur_CA', 'Couverture_des_interets',
       'CAF_sur_CA', 'Capacite_de_remboursement', 'Marge_EBE',
       'Resultat_courant_avant_impots_sur_CA',
       'Poids_BFR_exploitation_sur_CA_jours', 'Rotation_des_stocks_jours',
       'Credit_clients_Jours', 'Credit_fournisseurs_Jours', 'type_bilan',
       'confidentiality'],
      dtype='object')

In [44]:
df_financier.head(10)

,siren,date_cloture_exercice,Chiffre_d_affaires,Marge_brute,EBE,EBIT,Resultat_net,Taux_d_endettement,Ratio_de_liquidite,Ratio_de_vetuste,...,CAF_sur_CA,Capacite_de_remboursement,Marge_EBE,Resultat_courant_avant_impots_sur_CA,Poids_BFR_exploitation_sur_CA_jours,Rotation_des_stocks_jours,Credit_clients_Jours,Credit_fournisseurs_Jours,type_bilan,confidentiality
0,818842650,2019-12-31,0,0,-229,-229,-230,0.000,104.682,NaN,...,NaN,0.000,NaN,NaN,NaN,NaN,NaN,1482.969,S,Public
1,818845356,2022-12-31,0,0,-852,-1033,-1033,1.489,231.699,0.000,...,NaN,0.000,NaN,NaN,NaN,NaN,NaN,845.070,S,Public
2,818858086,2017-12-31,426247,426247,-5579,-7649,-8189,0.000,89.319,74.928,...,-1.269,0.000,-1.309,-1.963,-77.419,0.0,105.456,71.383,C,Public
3,818861965,2018-12-31,122080,117819,-9563,-12387,-12990,164.647,170.122,12.402,...,-8.317,-1.258,-7.833,-10.396,60.541,0.0,21.895,33.346,C,Public
4,818861965,2023-12-31,23320,23030,12014,2798,1767,3.778,284.314,NaN,...,47.097,0.186,51.518,11.998,866.316,0.0,1059.738,0.000,S,Public
5,818872707,2019-12-31,15447,15447,7519,7058,5999,26.420,1907.612,50.446,...,41.827,0.732,48.676,45.692,-11.979,0.0,0.000,25.253,C,Public
6,818905903,2023-12-31,0,0,0,0,15114,937.141,89.894,31.203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,Partiellement confidentiel
7,818917353,2017-12-31,0,0,-8349,-8832,0,1611.800,227.392,38.854,...,NaN,33.371,NaN,NaN,NaN,NaN,NaN,24.217,C,Public
8,818920076,2020-03-31,95267,95267,29978,29023,28466,2.184,970.001,16.483,...,29.936,0.127,30.573,29.554,82.806,0.0,102.149,41.617,C,Public
9,818964413,2022-12-31,3718438,3718438,177566,161863,121082,0.046,518.264,79.265,...,3.678,0.000,4.775,4.342,33.163,0.0,54.273,15.747,C,Public


In [45]:
#SUPPRESSION DES VARIABLES NON EXPLICATIVES

df_financier=df_financier.drop(["type_bilan","confidentiality"], axis=1)

In [46]:


df_financier['date_cloture_exercice'] = pd.to_datetime(df_financier['date_cloture_exercice'])
df_financier = df_financier.sort_values(['siren', 'date_cloture_exercice'])

In [47]:
#CONVERSION DATE BILAN EN ANNEE BILAN
df_financier['date_cloture_exercice'] = pd.to_datetime(df_financier['date_cloture_exercice'], errors='coerce')
df_financier['date_cloture_exercice'] = df_financier['date_cloture_exercice'].dt.year.astype(int)

In [48]:
#AJOUT DE LA COLONNE ANNEE DE CREATION DE L'ENTREPRISE

df_financier_avec_creation = df_financier.merge(
    df_adminetab[['siren', 'dateCreationUniteLegale','etatAdministratifUniteLegale']], 
    on='siren',
    how='left'  
)


In [49]:
#POURCENTAGE DE SURVIE

pourcentage = (df_financier_avec_creation['etatAdministratifUniteLegale'] == 1).mean()
print(f"Proportion de survie parmi les startups ayant déclaré un bilan financier : {pourcentage}")

prop = (df_adminetab["etatAdministratifUniteLegale"] == 1).mean()
print(f"Proportion de survie parmi toutes startups confondues :  {prop}")


Proportion de survie parmi les startups ayant déclaré un bilan financier : 0.8277414813745307
Proportion de survie parmi toutes startups confondues :  0.7405978818740125


In [50]:
#FILTRAGE DES LIGNES QUI CONCERNENT LES BILANS ENTRE LES ANNEES DE CREATION ET ANNEE DE CREATION +2 

del df_financier
gc.collect()

df_financier_filtre = df_financier_avec_creation[
    (df_financier_avec_creation['date_cloture_exercice'] >= df_financier_avec_creation['dateCreationUniteLegale']) &
    (df_financier_avec_creation['date_cloture_exercice'] <= df_financier_avec_creation['dateCreationUniteLegale'] + 2)
]


del df_financier_avec_creation
gc.collect()

0

In [51]:
# Vérification
df_financier_filtre.head(20)
df_financier_filtre.shape[0]
print(f"Nombre de startups ayant déclaré un bilan financier dans les 3 premières années de leur vie : {df_financier_filtre["siren"].nunique()}")

pourcentage = (df_financier_filtre['etatAdministratifUniteLegale'] == 1).mean() * 100
print(f"Pourcentage de survie parmi ces startups : {pourcentage}")

Nombre de startups ayant déclaré un bilan financier dans les 3 premières années de leur vie : 36388
Pourcentage de survie parmi ces startups : 76.77337470032435


In [52]:
#CONSTRUCTION DE LA VERSION 1 DU DATASET (ON COMPTE TOUTES LES STARTUPS ET ON MET UNE VARIABLE DE PRESENCE SUR LA DECLARATION D UN BILAN DANS LES 3 PREMIERES ANNEES)

sirens_filtre = set(df_financier_filtre["siren"].dropna().astype(str))

df_version1=df_adminetab.copy()


df_version1["aDéclaréBilan(s)FinancierSur3Ans"] = (
    df_version1["siren"]
    .astype(str)
    .isin(sirens_filtre)
    .astype(int)
)

df_version1.to_parquet('datasets_temp/dataset_sans_details_financiers1.parquet',engine='pyarrow',compression='snappy',index=False)


df_version1.head(10)

del sirens_filtre
gc.collect()

0

In [53]:
df_version1.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,aDéclaréBilan(s)FinancierSur3Ans
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0


In [54]:


df_adminetab.head(10)


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2


In [55]:


df_avec_bilan = df_version1[df_version1["aDéclaréBilan(s)FinancierSur3Ans"] == 1]

print(f"Nombre de startups avec bilan : {len(df_avec_bilan)}")
df_avec_bilan.head(10)

Nombre de startups avec bilan : 36388


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,aDéclaréBilan(s)FinancierSur3Ans
12747,480086784,2015,-1,0,3,1,2023,1,5710,7022,0,0,1,1
23272,514943323,2009,-1,0,3,1,2023,1,5710,7022,-1,0,3,1
24808,519715593,2010,2,2023,2,1,2023,1,5710,6209,-1,0,2,1
25111,520267881,2010,-1,0,2,1,2023,0,5710,7022,0,0,1,1
26416,522761139,2010,21,2023,5,1,2023,1,5710,7022,-1,0,8,1
26448,522807981,2010,12,2023,3,1,2023,1,5710,6399,-1,0,1,1
26996,523617173,2010,1,2023,6,1,2023,1,5710,6201,-1,0,3,1
27369,524157872,2010,-1,0,2,0,0,0,5710,7022,0,0,1,1
27624,524607868,2010,-1,0,4,1,2023,1,5710,6201,-1,0,2,1
27976,525268975,2010,-1,0,4,1,2023,0,5710,7022,-1,0,2,1


In [56]:
# #CONSTRUCTION DE LA VERSION 2 (ON NE RETIENT QUE LES ENTREPRISES QUI ONT DECLARE AU MOINS UN BILAN DANS LES 3 PREMIERES ANNEES)



df_financier_filtre.head(10)
df_financier_filtre.shape

(56728, 23)

In [57]:
#CALCUL DU POURCENTAGE DE VIDE POUR CHAQUE VARIABLE FINANCIERE
df_stats=df_financier_filtre.copy()


pourcentages_vide = df_stats.isna().mean() * 100
pourcentages_vide = pourcentages_vide.sort_values(ascending=False)

print("Classement des colonnes par taux de vide :")
print(pourcentages_vide)

df_stats.head(10)



Classement des colonnes par taux de vide :
Ratio_de_vetuste                        48.418770
Poids_BFR_exploitation_sur_CA           34.430969
Poids_BFR_exploitation_sur_CA_jours     34.430969
Rotation_des_stocks_jours               34.430969
CAF_sur_CA                              34.064307
Resultat_courant_avant_impots_sur_CA    33.817515
Marge_EBE                               33.817515
Credit_clients_Jours                    32.126992
Capacite_de_remboursement               17.538076
Couverture_des_interets                 17.504583
Credit_fournisseurs_Jours               15.530250
Ratio_de_liquidite                       4.634396
Taux_d_endettement                       0.174517
Autonomie_financiere                     0.081089
siren                                    0.000000
Resultat_net                             0.000000
EBIT                                     0.000000
EBE                                      0.000000
Marge_brute                              0.000000
Chiffre

,siren,date_cloture_exercice,Chiffre_d_affaires,Marge_brute,EBE,EBIT,Resultat_net,Taux_d_endettement,Ratio_de_liquidite,Ratio_de_vetuste,...,CAF_sur_CA,Capacite_de_remboursement,Marge_EBE,Resultat_courant_avant_impots_sur_CA,Poids_BFR_exploitation_sur_CA_jours,Rotation_des_stocks_jours,Credit_clients_Jours,Credit_fournisseurs_Jours,dateCreationUniteLegale,etatAdministratifUniteLegale
42971,480086784,2017,16927,16927,-33605,-33875,-21922,12.470,521.272,0.000,...,-135.417,-0.690,-198.529,-200.124,-356.789,0.000,0.000,43.591,2015,1
69544,514943323,2011,111048,56174,8907,-314,-1838,1310.938,146.013,66.939,...,6.502,4.997,8.021,-1.655,20.359,22.369,0.000,33.286,2009,1
72503,519715593,2012,149998,81719,32267,27064,22987,0.348,101.926,87.976,...,18.801,0.007,21.512,18.043,30.022,5.602,39.735,42.464,2010,1
73019,520267881,2012,37682,37682,-3997,-3665,-6761,0.000,38.141,89.354,...,-17.467,0.000,-10.607,-9.726,-177.764,0.000,0.000,52.993,2010,0
75241,522761139,2012,2216615,2216615,498608,497102,338041,13.778,298.840,9.377,...,15.097,0.239,22.154,22.113,39.616,0.000,78.778,21.579,2010,1
75277,522807981,2012,289372,275091,30471,3902,2447,777.829,76.959,64.803,...,10.587,1.199,10.473,1.024,-124.735,37.858,70.764,152.765,2010,1
76133,523617173,2012,155322,88096,-143840,-174688,212944,83.397,83.785,51.904,...,156.960,0.119,-92.608,-124.930,-116.213,7.751,354.581,102.188,2010,1
76838,524157872,2012,39327,39327,52,45,54,0.000,829.539,NaN,...,0.137,0.000,0.132,0.114,-4.238,0.000,0.000,73.204,2010,0
77215,524607868,2012,74920,73069,-13609,314,314,-0.986,16.776,58.341,...,-18.167,0.000,-18.165,0.419,-137.619,0.000,23.357,0.813,2010,1
77783,525268975,2012,281024,95221,34381,33058,26871,93.984,239.818,62.015,...,10.033,1.891,12.234,11.375,98.342,104.706,3.013,70.758,2010,0


In [58]:
#Permet de pouvoir reprendre le dataset à partir de ce point (et de ne pas rexecuter tout le code)
df=df_financier_filtre.copy()



In [59]:

#Taux de croissance annuel moyen (TCAM)
#Retourne 0 s'il y a une seule valeur, NaN si problème.

def tcam(vals): #prend les 3 valeurs de chaque startup

    
    #pas de croissance si une seule valeur au plus
    if len(vals) <= 1:
        return np.nan
    
    v_debut = vals[0]
    v_fin = vals[-1]
    n_periods = len(vals) - 1 #nombre d'intervalles entre les observations
    
    #cas où valeur initiale=0
    if v_debut == 0:
        return np.nan
    
    #cas où signes opposés
    if v_debut * v_fin < 0:
        return np.nan
    
    #valeurs conformes au calcul
    try:
        ratio = v_fin / v_debut
        return (ratio**(1/n_periods) - 1) * 100
    except:
        return np.nan




In [60]:
#CALCUL DES MOYENNES, ECARTS-TYPES ET TCAM DE CHAQUE STARTUP

colonnes_financieres = [c for c in df.columns 
                        if c not in ["siren", "date_cloture_exercice", 
                                     "dateCreationUniteLegale", "etatAdministratifUniteLegale"]]

resultats = []

for siren, groupe in df.groupby("siren"):
    ligne = {"siren": siren}

    groupe = groupe.sort_values('date_cloture_exercice')
    
    for col in colonnes_financieres:
        # Prendre les 3 premières valeurs non-nulles
        vals = groupe[col].dropna().values[:3]
        
        ligne[f"{col}_tcam"] = tcam(vals)
        
        #Moyenne (calculé si au moins une valeur  existante)
        if(len(vals)>=1):
            ligne[f"{col}_moyen"] = np.mean(vals)
        else:
            ligne[f"{col}_moyen"] = np.nan

        #STD (calculée si au moins 2 valeurs existantes)
        if len(vals) >= 2:
            ligne[f"{col}_std"] = np.std(vals)
        else:
            ligne[f"{col}_std"] = np.nan


    resultats.append(ligne)

df_financier = pd.DataFrame(resultats)


In [61]:
df_financier.head(10)
df_financier.isna().sum()


siren                                             0
Chiffre_d_affaires_tcam                       25486
Chiffre_d_affaires_moyen                          0
Chiffre_d_affaires_std                        19657
Marge_brute_tcam                              25464
Marge_brute_moyen                                 0
Marge_brute_std                               19657
EBE_tcam                                      25307
EBE_moyen                                         0
EBE_std                                       19657
EBIT_tcam                                     25315
EBIT_moyen                                        0
EBIT_std                                      19657
Resultat_net_tcam                             24653
Resultat_net_moyen                                0
Resultat_net_std                              19657
Taux_d_endettement_tcam                       26044
Taux_d_endettement_moyen                         54
Taux_d_endettement_std                        19690
Ratio_de_liq

In [62]:

# Pour les TCAM, peut-être utiliser 0 au lieu de la médiane
cols_tcam = [col for col in df_financier.columns if col.endswith('_tcam')]
for col in cols_tcam:
    df_financier[col] = df_financier[col].fillna(0)

In [63]:
df_financier.isna().sum()

siren                                             0
Chiffre_d_affaires_tcam                           0
Chiffre_d_affaires_moyen                          0
Chiffre_d_affaires_std                        19657
Marge_brute_tcam                                  0
Marge_brute_moyen                                 0
Marge_brute_std                               19657
EBE_tcam                                          0
EBE_moyen                                         0
EBE_std                                       19657
EBIT_tcam                                         0
EBIT_moyen                                        0
EBIT_std                                      19657
Resultat_net_tcam                                 0
Resultat_net_moyen                                0
Resultat_net_std                              19657
Taux_d_endettement_tcam                           0
Taux_d_endettement_moyen                         54
Taux_d_endettement_std                        19690
Ratio_de_liq

In [64]:
#FUSION DES DATASETS
df_complete = df_financier.merge(
    df_adminetab,
    on='siren',
    how='left'
)


In [65]:

df_complete.head(10)

,siren,Chiffre_d_affaires_tcam,Chiffre_d_affaires_moyen,Chiffre_d_affaires_std,Marge_brute_tcam,Marge_brute_moyen,Marge_brute_std,EBE_tcam,EBE_moyen,EBE_std,...,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements
0,480086784,0.0,16927.0,NaN,0.0,16927.0,NaN,0.0,-33605.0,NaN,...,0,3,1,2023,1,5710,7022,0,0,1
1,514943323,0.0,111048.0,NaN,0.0,56174.0,NaN,0.0,8907.0,NaN,...,0,3,1,2023,1,5710,7022,-1,0,3
2,519715593,0.0,149998.0,NaN,0.0,81719.0,NaN,0.0,32267.0,NaN,...,2023,2,1,2023,1,5710,6209,-1,0,2
3,520267881,0.0,37682.0,NaN,0.0,37682.0,NaN,0.0,-3997.0,NaN,...,0,2,1,2023,0,5710,7022,0,0,1
4,522761139,0.0,2216615.0,NaN,0.0,2216615.0,NaN,0.0,498608.0,NaN,...,2023,5,1,2023,1,5710,7022,-1,0,8
5,522807981,0.0,289372.0,NaN,0.0,275091.0,NaN,0.0,30471.0,NaN,...,2023,3,1,2023,1,5710,6399,-1,0,1
6,523617173,0.0,155322.0,NaN,0.0,88096.0,NaN,0.0,-143840.0,NaN,...,2023,6,1,2023,1,5710,6201,-1,0,3
7,524157872,0.0,39327.0,NaN,0.0,39327.0,NaN,0.0,52.0,NaN,...,0,2,0,0,0,5710,7022,0,0,1
8,524607868,0.0,74920.0,NaN,0.0,73069.0,NaN,0.0,-13609.0,NaN,...,0,4,1,2023,1,5710,6201,-1,0,2
9,525268975,0.0,281024.0,NaN,0.0,95221.0,NaN,0.0,34381.0,NaN,...,0,4,1,2023,0,5710,7022,-1,0,2


In [66]:
#REARRANGER LES COLONNES


cols_admin = df_adminetab.columns.tolist() 
cols_financier = [c for c in df_complete.columns if c not in cols_admin]
df_complete = df_complete[cols_admin + cols_financier]

In [67]:
df_complete.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,...,Poids_BFR_exploitation_sur_CA_jours_std,Rotation_des_stocks_jours_tcam,Rotation_des_stocks_jours_moyen,Rotation_des_stocks_jours_std,Credit_clients_Jours_tcam,Credit_clients_Jours_moyen,Credit_clients_Jours_std,Credit_fournisseurs_Jours_tcam,Credit_fournisseurs_Jours_moyen,Credit_fournisseurs_Jours_std
0,480086784,2015,-1,0,3,1,2023,1,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,43.591,NaN
1,514943323,2009,-1,0,3,1,2023,1,5710,7022,...,NaN,0.0,22.369,NaN,0.0,0.000,NaN,0.0,33.286,NaN
2,519715593,2010,2,2023,2,1,2023,1,5710,6209,...,NaN,0.0,5.602,NaN,0.0,39.735,NaN,0.0,42.464,NaN
3,520267881,2010,-1,0,2,1,2023,0,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,52.993,NaN
4,522761139,2010,21,2023,5,1,2023,1,5710,7022,...,NaN,0.0,0.000,NaN,0.0,78.778,NaN,0.0,21.579,NaN
5,522807981,2010,12,2023,3,1,2023,1,5710,6399,...,NaN,0.0,37.858,NaN,0.0,70.764,NaN,0.0,152.765,NaN
6,523617173,2010,1,2023,6,1,2023,1,5710,6201,...,NaN,0.0,7.751,NaN,0.0,354.581,NaN,0.0,102.188,NaN
7,524157872,2010,-1,0,2,0,0,0,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,73.204,NaN
8,524607868,2010,-1,0,4,1,2023,1,5710,6201,...,NaN,0.0,0.000,NaN,0.0,23.357,NaN,0.0,0.813,NaN
9,525268975,2010,-1,0,4,1,2023,0,5710,7022,...,NaN,0.0,104.706,NaN,0.0,3.013,NaN,0.0,70.758,NaN


In [68]:
df_complete.shape

(36388, 70)

In [69]:
df_complete.isna().sum()

siren                                  0
dateCreationUniteLegale                0
trancheEffectifsUniteLegale            0
anneeEffectifsUniteLegale              0
nombrePeriodesUniteLegale              0
                                   ...  
Credit_clients_Jours_moyen         10530
Credit_clients_Jours_std           25813
Credit_fournisseurs_Jours_tcam         0
Credit_fournisseurs_Jours_moyen     5439
Credit_fournisseurs_Jours_std      22450
Length: 70, dtype: int64

In [70]:

# # Sauvegarder pour reprendre plus tard sans tout executer
df_complete.to_parquet('datasets_temp/dataset_avec_details_financiers1.parquet',
                 engine='pyarrow', compression='snappy', index=False)


In [71]:
df_sans_details_financiers=pd.read_parquet("datasets_temp/dataset_sans_details_financiers1.parquet")
df_avec_details_financiers=pd.read_parquet("datasets_temp/dataset_avec_details_financiers1.parquet")

In [72]:

del df
del df_complete
del df_stats
del df_financier_filtre
del df_financier

gc.collect()

0

In [73]:
df_avec_details_financiers.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,...,Poids_BFR_exploitation_sur_CA_jours_std,Rotation_des_stocks_jours_tcam,Rotation_des_stocks_jours_moyen,Rotation_des_stocks_jours_std,Credit_clients_Jours_tcam,Credit_clients_Jours_moyen,Credit_clients_Jours_std,Credit_fournisseurs_Jours_tcam,Credit_fournisseurs_Jours_moyen,Credit_fournisseurs_Jours_std
0,480086784,2015,-1,0,3,1,2023,1,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,43.591,NaN
1,514943323,2009,-1,0,3,1,2023,1,5710,7022,...,NaN,0.0,22.369,NaN,0.0,0.000,NaN,0.0,33.286,NaN
2,519715593,2010,2,2023,2,1,2023,1,5710,6209,...,NaN,0.0,5.602,NaN,0.0,39.735,NaN,0.0,42.464,NaN
3,520267881,2010,-1,0,2,1,2023,0,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,52.993,NaN
4,522761139,2010,21,2023,5,1,2023,1,5710,7022,...,NaN,0.0,0.000,NaN,0.0,78.778,NaN,0.0,21.579,NaN
5,522807981,2010,12,2023,3,1,2023,1,5710,6399,...,NaN,0.0,37.858,NaN,0.0,70.764,NaN,0.0,152.765,NaN
6,523617173,2010,1,2023,6,1,2023,1,5710,6201,...,NaN,0.0,7.751,NaN,0.0,354.581,NaN,0.0,102.188,NaN
7,524157872,2010,-1,0,2,0,0,0,5710,7022,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,73.204,NaN
8,524607868,2010,-1,0,4,1,2023,1,5710,6201,...,NaN,0.0,0.000,NaN,0.0,23.357,NaN,0.0,0.813,NaN
9,525268975,2010,-1,0,4,1,2023,0,5710,7022,...,NaN,0.0,104.706,NaN,0.0,3.013,NaN,0.0,70.758,NaN


In [74]:
df_sans_details_financiers.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,aDéclaréBilan(s)FinancierSur3Ans
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0


In [75]:
df_sans_details_financiers['dateCreationUniteLegale'].describe()

count    285441.000000
mean       2018.136519
std           6.964259
min        1900.000000
25%        2016.000000
50%        2020.000000
75%        2023.000000
max        2026.000000
Name: dateCreationUniteLegale, dtype: float64

In [76]:
df_avec_details_financiers['dateCreationUniteLegale'].describe()

count    36388.000000
mean      2018.387765
std          2.830421
min       2009.000000
25%       2016.000000
50%       2018.000000
75%       2021.000000
max       2025.000000
Name: dateCreationUniteLegale, dtype: float64

In [77]:
#pour montrer que le nombre d'établissements ne spoil pas la survie de la startup

print((df_sans_details_financiers['nbEtablissements']==0).sum())
print((df_avec_details_financiers['nbEtablissements']== 0).sum())



0
0


In [78]:
print((df_sans_details_financiers['dateCreationUniteLegale']>= 2009).sum())
print((df_avec_details_financiers['dateCreationUniteLegale']>= 2009).sum())

264249
36388


In [79]:
#garder les entreprises créées au plus tôt en 2009 pour éliminer
#les entreprises trop anciennes (par exemple 1970) tout en ne perdant pas d'instances
#parmi celles qui ont déclaré un bilan financier dans leurs trois premières années de vie.


df_avec_details_financiers=df_avec_details_financiers[df_avec_details_financiers['dateCreationUniteLegale']>=2009]
df_sans_details_financiers=df_sans_details_financiers[df_sans_details_financiers['dateCreationUniteLegale']>=2009]

In [80]:
proportion_survie_sans_details = (df_sans_details_financiers['etatAdministratifUniteLegale'] == 1).mean() 
print(f"Proportion de survie parmi toutes les startups confondues : {proportion_survie_sans_details}")


proportion_survie_avec_details = (df_avec_details_financiers['etatAdministratifUniteLegale'] == 1).mean() 
print(f"Proportion de survie parmi les startups ayant déclaré un bilan financier dans les 3 premières années de leur existence : {proportion_survie_avec_details}")

Proportion de survie parmi toutes les startups confondues : 0.7465685773645312
Proportion de survie parmi les startups ayant déclaré un bilan financier dans les 3 premières années de leur existence : 0.7539848301637903


In [81]:

#Observation sur la répartition des startups dans le datast selon leur année de création 
print(df_avec_details_financiers['dateCreationUniteLegale'].describe())
print(df_sans_details_financiers['dateCreationUniteLegale'].describe())

count    36388.000000
mean      2018.387765
std          2.830421
min       2009.000000
25%       2016.000000
50%       2018.000000
75%       2021.000000
max       2025.000000
Name: dateCreationUniteLegale, dtype: float64
count    264249.000000
mean       2019.576596
std           4.200730
min        2009.000000
25%        2017.000000
50%        2020.000000
75%        2023.000000
max        2026.000000
Name: dateCreationUniteLegale, dtype: float64


In [82]:
df_sans_details_financiers=df_sans_details_financiers.drop(["dateCreationUniteLegale"],axis=1)
df_avec_details_financiers=df_avec_details_financiers.drop(["dateCreationUniteLegale"],axis=1)

In [83]:
df_sans_details_financiers.var()

siren                                  1.048377e+16
trancheEffectifsUniteLegale            5.599622e+00
anneeEffectifsUniteLegale              6.700093e+05
nombrePeriodesUniteLegale              9.675682e-01
categorieEntreprise                    2.635660e-01
anneeCategorieEntreprise               9.625506e+05
etatAdministratifUniteLegale           1.892047e-01
categorieJuridiqueUniteLegale          0.000000e+00
activitePrincipaleUniteLegale          2.052288e+05
economieSocialeSolidaireUniteLegale    1.704007e-01
societeMissionUniteLegale              1.562362e-01
nbEtablissements                       7.980195e-01
aDéclaréBilan(s)FinancierSur3Ans       1.187417e-01
dtype: float64

In [84]:
df_avec_details_financiers.var()

siren                              2.468446e+15
trancheEffectifsUniteLegale        1.527636e+01
anneeEffectifsUniteLegale          9.359101e+05
nombrePeriodesUniteLegale          1.211760e+00
categorieEntreprise                2.281899e-01
                                       ...     
Credit_clients_Jours_moyen         1.098621e+08
Credit_clients_Jours_std           2.151265e+08
Credit_fournisseurs_Jours_tcam     2.827669e+06
Credit_fournisseurs_Jours_moyen    3.193283e+07
Credit_fournisseurs_Jours_std      7.141303e+07
Length: 69, dtype: float64

In [85]:
#DETECTION DE VARIABLES A VARIANCE NULLE

variances = df_avec_details_financiers.select_dtypes(include="number").var()
variances_nulles = variances[variances==0]

variances_nulles

categorieJuridiqueUniteLegale    0.0
dtype: float64

In [86]:
df_sans_details_financiers=df_sans_details_financiers.drop(["categorieJuridiqueUniteLegale"],axis=1) 
#car c'est tout le temps la même valeur, donc pas de variance
df_avec_details_financiers=df_avec_details_financiers.drop(["categorieJuridiqueUniteLegale"],axis=1)

In [87]:
#SAUVEGARDE DES FICHIERS FINAUX

df_sans_details_financiers.to_parquet('datasets_finaux/dataset_sans_details_financiers.parquet',engine='pyarrow',compression='snappy',index=False)
df_avec_details_financiers.to_parquet('datasets_finaux/dataset_avec_details_financiers.parquet',engine='pyarrow',compression='snappy',index=False)

In [88]:
#SUPPRESSION DES FICHIERS TEMPORAIRES

fichiers=["datasets_temp/bilan_admin_v2.parquet",
          "datasets_temp/bilan_admin_v3.parquet",
          "datasets_temp/bilan_admin_v4.parquet",
          "datasets_temp/bilan_etablissement_v2.parquet",
          "datasets_temp/bilan_etablissement_v3.parquet",
          "datasets_temp/bilan_financier_v2.parquet",
          "datasets_temp/bilan_adminetab.parquet",
          "datasets_temp/dataset_sans_details_financiers1.parquet",
          "datasets_temp/dataset_avec_details_financiers1.parquet"]


for fichier in fichiers:
    os.remove(fichier)

In [89]:
os.rmdir("datasets_temp")

In [90]:

del df_avec_details_financiers
del df_sans_details_financiers


gc.collect()

0